<a href="https://colab.research.google.com/github/tranvien98/LLM-learn/blob/main/LLM_settings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Giải thích các thiết lập của mô hình ngôn ngữ lớn (LLM), bao gồm nhiệt độ (temperature), Top P, độ dài tối đa (max length), dãy dừng (stop sequences), phạt tần suất (frequency penalty) và phạt sự hiện diện (presence penalty). Các thiết lập này ảnh hưởng đến sự ngẫu nhiên, tính đa dạng và độ dài của đầu ra LLM. Hướng dẫn cũng đề cập đến việc không nên thay đổi cả nhiệt độ và Top P hoặc phạt tần suất và phạt sự hiện diện cùng lúc. Cuối cùng, tài liệu liệt kê nhiều ứng dụng và mô hình LLM khác nhau.

In [1]:
# install package
%pip install -U langchain-ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [13]:
import os
os.environ["LLM_URL"] = "https://a066-34-169-30-144.ngrok-free.app"

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(base_url=os.environ.get("LLM_URL"), model="llama3.1:8b")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

'To answer what LangChain is, let\'s break it down into simpler concepts and then build upon those to arrive at a comprehensive understanding.\n\n1. **Understanding the Basic Components:**\n   - The term "Lang" in LangChain likely stands for language, indicating that this technology has something to do with linguistic or computational processes related to language.\n   - The suffix "-Chain" implies a series of connected things or events, suggesting a concept that involves a sequence of actions or components linked together.\n\n2. **Analyzing the Purpose:**\n   Given that LangChain involves language and sequences of operations, it likely pertains to natural language processing (NLP) or artificial intelligence (AI). The primary goal in AI is often to create systems that can understand and generate human-like text based on given inputs, such as conversational dialogue.\n\n3. **Considering the Potential Applications:**\n   - In the context of NLP, LangChain might be related to chatbots, la

## Tham số temperature
Tham số này ảnh hướng đến việc chọn token tiếp theo. Trong các mô hình AI thì token tiếp theo luôn được chọn với xác suất cao nhât. Điều chỉnh temperature thêm tính ngẫu nhiên cho việc chọn các token.
- Temperature thấp: Tạo ra các kết quả chắc chắn. mô hình luôn chọn các token có xác suất cao nhất.
- Temperature cao: Tạo ra kết quả có sự đa dạng hơn. => Thích hợp cho các task cần tính sáng tạo cao.

p_i = softmax(x_i/t)

công thức gốc: p_i = softmax(x_i)

In [8]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(base_url=os.environ.get("LLM_URL"), model="llama3.1:8b", temperature=0)

model.invoke("Thực hiện làm bài thơ về Hà Nội khoảng 4 câu?")

'Hà Nội - thành phố của tình yêu\nLòng người yêu thương, tràn đầy niềm vui\nCảnh đẹp như tranh vẽ, không nơi nào sánh được\nHà Nội - nơi tôi yêu thương nhất trên đời.'

In [9]:
model.invoke("Thực hiện làm bài thơ về Hà Nội khoảng 4 câu?")

'Hà Nội - thành phố của tình yêu\nLòng người yêu thương, tràn đầy niềm vui\nCảnh đẹp như tranh vẽ, không nơi nào sánh được\nHà Nội - nơi tôi yêu thương nhất trên đời.'

In [10]:
model = OllamaLLM(base_url=os.environ.get("LLM_URL"), model="llama3.1:8b", temperature=1)

model.invoke("Thực hiện làm bài thơ về Hà Nội khoảng 4 câu?")

'Đường cũ huyền bí\nVề nhà tò mò đường đến trường\nTrên phố Bát Đàn\nSợi sẹo của thời gian còn sót lại \n\n(Lời nho đề đựng cảm nhận của tác giả khi đi đến Hà Nội)'

In [11]:
model.invoke("Thực hiện làm bài thơ về Hà Nội khoảng 4 câu?")

'- Núi Thăng Long cao vang trập trùng, đằng trước là hồ Tây lớn\n- Lâu Bờ Động và Văn Miếu cùng đứng yên.\n- Lá đỏ xuân rơi, phố cũ chìm trong màu tàn\n- Trong khung cảnh hối hả ồn ào của thành quê.'

=> Nhận xét: Khi temperture=1 thì 2 câu trả lời in ra là khác nhau


## Tham số top_k

Để tránh việc tốn chi phí tính toán. sau khi tính softmax chọn ra top k giá trị. Bao gồm 2 quá trình tính softmax và nomalization lại giá trị(vì từ vựng của mô hình ngôn ngữ có thể rất lớn nên tốn chi phí tính toán).

Top_k phải nhỏ hơn số lượng từ vựng của model. Nếu top k quá nhỏ thì mô hình trả lời sẽ kém đa dạng.

In [14]:
model = OllamaLLM(base_url=os.environ.get("LLM_URL"), model="llama3.1:8b", temperature=1, top_k=20)

model.invoke("Thực hiện làm bài thơ về Hà Nội khoảng 4 câu?")

'Hà Nội xưa nay vẫn là thành phố cổ kính\nHàng rong và chợ đêm rất nổi tiếng ở Hà Nội\n\nBánh bè, bánh cuốn, phở hay mắm tôm đều là những món ăn ngon nổi tiếng.\n\nTừ phố cổ đến Hoàn Kiếm, Hà Nội thật sự không thể thiếu.'

In [15]:
model = OllamaLLM(base_url=os.environ.get("LLM_URL"), model="llama3.1:8b", temperature=1, top_k=5)

model.invoke("Thực hiện làm bài thơ về Hà Nội khoảng 4 câu?")

'- Hà Nội - thành phố xưa cũ\n- Cốm, dừa xiêm là đặc sản của nơi đây\n- Làng Thợ Đỏ - nơi lưu giữ những truyền thống dân tộc\n- Hà Nội không thể thiếu phố cổ.'

## Tham số top_p

Trong top_p. model tính tổng các xác suất, lấy các giá trị tại nơi mà tổng <= top_p.

ví dụ yes:0.6, maybe:0.3, no: 0.1. top_p = 0.9 => yes mà maybe được xem sét

top_p không làm giảm quá trình tính toán softmax. Lợi ích  tập hợp vào các từ liên quan đến context nhất.

Lưu ý: Không nên sử dụng top_p và temperature cùng 1 lúc.

## Tham số Stopping condition
Điều kiện dùng có thể là số lượng token hoặc gặp 1 kí tự nào đó. => Kỹ thuật này làm giảm chi phí tính toán.
